# Creating our own prediction model void of predictor variables from data

In [135]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(6,6)})
%matplotlib inline
import warnings
warnings.simplefilter("ignore")

In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
df = pd.read_csv("data/compas-scores-two-years.csv")
df.shape
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [137]:
pd.set_option('max_columns',53)

df = df.drop(['first','last','decile_score','c_offense_date',
         'r_offense_date','is_violent_recid','type_of_assessment',
         'decile_score.1','score_text','v_type_of_assessment',
         'v_decile_score',"v_score_text",'v_screening_date',
         'in_custody','out_custody','c_case_number','c_days_from_compas',
         'is_recid','r_case_number','r_days_from_arrest','r_jail_in','r_jail_out',
         'violent_recid','vr_case_number','vr_charge_degree','vr_offense_date',
         'screening_date','days_b_screening_arrest','c_arrest_date','name',
         'compas_screening_date','id','dob','c_jail_in','c_jail_out', 'age_cat',
         'priors_count.1', 'decile_score.1', 'r_charge_degree'],axis=1)
df.shape

(7214, 15)

In [138]:
nums = df.select_dtypes(np.number).fillna(0)

In [139]:
print(nums.shape)
nums.head()
nums['time_in']= nums['end']-nums['start']
nums.drop(['start','end'],axis=1)
nums.reset_index()
nums.tail(20)

(7214, 9)


,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,start,end,event,two_year_recid,time_in
7194,30,0,0,0,2,1,351,1,1,350
7195,35,0,0,0,8,62,393,1,1,331
7196,43,0,0,0,0,0,938,0,0,938
7197,28,0,0,0,1,0,764,0,0,764
7198,32,0,0,0,4,0,653,0,0,653
7199,23,0,0,0,0,2,913,0,0,911
7200,20,0,0,0,0,4,22,1,1,18
7201,30,0,1,0,19,9,144,1,1,135
7202,26,0,0,0,0,173,322,0,0,149
7203,44,0,0,0,0,0,887,0,0,887


In [140]:
obj = df.select_dtypes(object)
obj = obj.drop(['r_charge_desc','c_charge_desc','vr_charge_desc'],axis=1)
obj

,sex,race,c_charge_degree
0,Male,Other,F
1,Male,African-American,F
2,Male,African-American,F
3,Male,African-American,F
4,Male,Other,F
...,...,...,...
7209,Male,African-American,F
7210,Male,African-American,F
7211,Male,Other,F
7212,Female,African-American,M


In [141]:
obj_binary = pd.get_dummies(obj,
                            dtype="int64",
                            drop_first = False
).reset_index()

In [142]:
obj_binary.shape

(7214, 11)

In [143]:
df_clean = pd.concat([obj_binary, nums], axis = 1)

In [144]:
df_clean.head()
df_clean = df_clean.drop(['start','end','event'],axis=1)


In [145]:
df_clean.head()

,index,sex_Female,sex_Male,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,c_charge_degree_F,c_charge_degree_M,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,two_year_recid,time_in
0,0,0,1,0,0,0,0,0,1,1,0,69,0,0,0,0,0,327
1,1,0,1,1,0,0,0,0,0,1,0,34,0,0,0,0,1,150
2,2,0,1,1,0,0,0,0,0,1,0,24,0,0,1,4,1,63
3,3,0,1,1,0,0,0,0,0,1,0,23,0,1,0,1,0,1174
4,4,0,1,0,0,0,0,0,1,1,0,43,0,0,0,2,0,1102


In [146]:
df_clean = df_clean.drop(['race_Asian','race_Hispanic','race_Native American','race_Other'],axis=1)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
df_clean.head()

,index,sex_Female,sex_Male,race_African-American,race_Caucasian,c_charge_degree_F,c_charge_degree_M,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,two_year_recid,time_in
0,0,0,1,0,0,1,0,69,0,0,0,0,0,327
1,1,0,1,1,0,1,0,34,0,0,0,0,1,150
2,2,0,1,1,0,1,0,24,0,0,1,4,1,63
3,3,0,1,1,0,1,0,23,0,1,0,1,0,1174
4,4,0,1,0,0,1,0,43,0,0,0,2,0,1102


In [147]:
aay = df_clean[df_clean['race_African-American'] == 1].two_year_recid
aaX = df_clean[df_clean['race_African-American'] == 1].drop('two_year_recid',axis=1)
X_train, X_test, y_train, y_test = train_test_split(aaX, aay,
                                                    test_size=0.2,
                                                    random_state=42)

In [148]:
model = LogisticRegression(class_weight = 'balanced')

In [149]:
from sklearn.model_selection import cross_val_score
cross_val_score(
    model, aaX, aay, scoring = "precision", cv = 10
).mean()

0.8600995538088755

In [150]:
cross_val_score(
    model, aaX, aay, scoring = "recall", cv = 10
).mean()

0.903213006337834

In [151]:
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [152]:
predictions = model.predict(X_test)
predct_prob  = model.predict_proba(X_test)
predct_prob = predct_prob[:,0]
true_classes = y_test

In [153]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_classes, predictions)

array([[314,  57],
       [ 24, 345]])

In [154]:
X_test.count()

index                    740
sex_Female               740
sex_Male                 740
race_African-American    740
race_Caucasian           740
c_charge_degree_F        740
c_charge_degree_M        740
age                      740
juv_fel_count            740
juv_misd_count           740
juv_other_count          740
priors_count             740
time_in                  740
dtype: int64

In [155]:
print('The false positive rate for African-Americans is: ' + str(57/740))

The false positive rate for African-Americans is: 0.07702702702702703


In [156]:
print('The false negative rate for African-Americans is: ' + str(24/740))

The false negative rate for African-Americans is: 0.032432432432432434


In [157]:


df_clean.head(20)

,index,sex_Female,sex_Male,race_African-American,race_Caucasian,c_charge_degree_F,c_charge_degree_M,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,two_year_recid,time_in
0,0,0,1,0,0,1,0,69,0,0,0,0,0,327
1,1,0,1,1,0,1,0,34,0,0,0,0,1,150
2,2,0,1,1,0,1,0,24,0,0,1,4,1,63
3,3,0,1,1,0,1,0,23,0,1,0,1,0,1174
4,4,0,1,0,0,1,0,43,0,0,0,2,0,1102
5,5,0,1,0,0,0,1,44,0,0,0,0,0,852
6,6,0,1,0,1,1,0,41,0,0,0,14,1,35
7,7,0,1,0,0,1,0,43,0,0,0,3,0,265
8,8,1,0,0,1,0,1,39,0,0,0,0,0,745
9,9,0,1,0,1,1,0,21,0,0,0,1,1,428


In [158]:
cy = df_clean[df_clean['race_Caucasian'] == 1].two_year_recid
cX = df_clean[df_clean['race_Caucasian'] == 1].drop('two_year_recid',axis=1)
cX_train, cX_test, cy_train, cy_test = train_test_split(cX, cy,
                                                    test_size=0.2,
                                                    random_state=42)

In [159]:
model = LogisticRegression(class_weight = 'balanced')

In [160]:
from sklearn.model_selection import cross_val_score
cross_val_score(
    model, cX, cy, scoring = "precision", cv = 10
).mean()


0.8084026903819576

In [161]:
cross_val_score(
    model, cX, cy, scoring = "recall", cv = 10
).mean()

0.9058204467353953

In [162]:
model.fit(cX_train, cy_train)
predictions = model.predict(cX_test)
predct_prob  = model.predict_proba(cX_test)
predct_prob = predct_prob[:,0]
true_classes = cy_test

In [163]:
confusion_matrix(true_classes, predictions)

array([[238,  43],
       [ 22, 188]])

In [164]:
cX_test.count()

index                    491
sex_Female               491
sex_Male                 491
race_African-American    491
race_Caucasian           491
c_charge_degree_F        491
c_charge_degree_M        491
age                      491
juv_fel_count            491
juv_misd_count           491
juv_other_count          491
priors_count             491
time_in                  491
dtype: int64

In [165]:
print('The false positive rate for Caucasians is: ' + str(43/491))

The false positive rate for Caucasians is: 0.08757637474541752


## Running two different Cross value scores with caucasian and African American, we can see that models can be made where race is not a factor